# 라이브러리 로드 및 데이터 로드

In [ ]:
# 맞춤법 검사 라이브러리 py-hanspell
# # !pip install git+https://github.com/ssut/py-hanspell.git

In [1]:
# 라이브러리 로드
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

# 전처리
import re
from hanspell import spell_checker
from tqdm import tqdm
from konlpy.tag import Okt
from collections import Counter

# 바이너리 파일 저장 및 로드
import pickle

In [ ]:
# 데이터셋 로드
# root_dir = "data"

# df = pd.read_parquet(f"{root_dir}/all_append_csv_12_26_1200i.gzip")
# display(df.head())
# df.shape

In [ ]:
# 발라드만 떼오기
# df_ballad = df[df["장르"] == "발라드"].copy()
# df_ballad.shape

In [2]:
# 현재 경로 확인
%pwd

'C:\\Users\\JongHyun_Moon\\Jupyter_Python\\LikeLion\\project\\eda'

In [3]:
# 디렉토리 이동
%cd ..

C:\Users\JongHyun_Moon\Jupyter_Python\LikeLion\project


In [4]:
%pwd

'C:\\Users\\JongHyun_Moon\\Jupyter_Python\\LikeLion\\project'

In [5]:
root_dir = "data/data_ballad"
df_ballad = pd.read_csv(f"{root_dir}/melon_ballad_1_15000.csv", parse_dates=["발매일"])
display(df_ballad.head())
df_ballad.shape

,제목,가사,가수,발매일,좋아요수,장르
0,Monologue,다 잊었다는 거짓말\r\n또 해 버렸죠\r\n내 마음에 그대란 사람\r\n없다고 했...,테이,2022.09.18,"66,062",발라드
1,너의 모든 순간,이윽고 내가 한눈에\r\n너를 알아봤을 때\r\n모든 건 분명 달라지고 있었어\r\...,성시경,2014.02.12,"243,092","발라드, 국내드라마"
2,잘가요,미안해 마요 이제야 난 깨달아요\r\n내 절대 그대 짝이 아님을\r\n괜찮을게요 영...,주호,2022.12.04,"18,702",발라드
3,사랑은 늘 도망가,눈물이 난다 이 길을 걸으면\r\n그 사람 손길이 자꾸 생각이 난다\r\n붙잡지 못...,임영웅,2021.10.11,"185,585","발라드, 국내드라마"
4,해요 (2022),그녀와 나는요 그땐 참 어렸어요\r\n많이 사랑했고 때론 많이 다퉜었죠\r\n지금 ...,#안녕,2022.06.07,"63,808","발라드, 인디음악"


(13346, 6)

# 컬럼 전처리

## 데이터 타입 변경

In [ ]:
# info
df_ballad.info()

In [ ]:
# 발매일 없는 데이터는 na로 처리
df_ballad["발매일"] = pd.to_datetime(df_ballad["발매일"], errors="coerce")

# 발매일 없는 데이터는 채울까 ? drop 할까?
df_ballad = df_ballad.dropna(subset=["발매일"])

In [ ]:
# 좋아요 수 int 타입으로 변경 -> concat할 때 처리함
# df_ballad["좋아요수"] = df_ballad["좋아요수"].str.replace(",", "")
# df_ballad["좋아요수"] = df_ballad["좋아요수"].astype(int)

In [ ]:
# info
df_ballad.info()

## 장르 구분

In [ ]:
# 장르 구분
genre_split = df_ballad["장르"].str.split(",")
df_ballad["장르1"] = genre_split.str.get(0)
df_ballad["장르2"] = genre_split.str.get(1)
df_ballad["장르3"] = genre_split.str.get(2)
df_ballad["장르4"] = genre_split.str.get(3)

In [ ]:
# 장르 분포 확인
# 장르는 한 곡에 최대 4개까지 있으나 양이 적어서 필요한 컬럼만 사용
display(df_ballad["장르1"].value_counts())
print("-"*30)
display(df_ballad["장르2"].value_counts())
print("-"*30)
display(df_ballad["장르3"].value_counts())
print("-"*30)
display(df_ballad["장르4"].value_counts())

## 날짜, 요일 추가
* 일단 다각면에서 분석하고자 다 만들어두긴 했으나.. 연도와 월 빼고는 쓸모가 없을 듯 싶다

In [ ]:
# 날짜, 요일 추가
df_ballad["년"] = df_ballad["발매일"].dt.year
df_ballad["월"] = df_ballad["발매일"].dt.month
# df_ballad["일"] = df_ballad["발매일"].dt.day
# df_ballad["요일"] = df_ballad["발매일"].dt.dayofweek

In [ ]:
# 계절 추가
df_ballad.loc[df_ballad["월"].isin([12,1,2]), "계절"] = "겨울"
df_ballad.loc[df_ballad["월"].isin([3,4,5]), "계절"] = "봄"
df_ballad.loc[df_ballad["월"].isin([6,7,8]), "계절"] = "여름"
df_ballad.loc[df_ballad["월"].isin([9,10,11]), "계절"] = "가을"

In [ ]:
# 연도별 노래 개수
df_ballad["년"].value_counts()

In [ ]:
# 계절별 노래 개수
df_ballad["계절"].value_counts()

In [ ]:
df_ballad.info()

# EDA

## 중복행 제거

In [ ]:
# 중복행 한 번 더 확인
df_ballad[df_ballad.duplicated(keep=False)]

In [ ]:
# 중복행 drop
print(df_ballad.shape)
df_ballad = df_ballad.drop_duplicates()
df_ballad.shape

## 가사 전처리

In [ ]:
# 필요 없는 단어 제거
df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("1절", "", x))
df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("2절", "", x))
df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("브리지", "", x).strip())

In [ ]:
# 이외 기본적인 전처리
df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("\s{2,}", " ", x)) # 공백 2회 이상 제거
df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("\n", " ", x)) # 개행문자 제거

## 시각화

### 가수별 분석

In [ ]:
# 가수별 곡 수
df_singer_soundtrack = df_ballad.groupby("가수").agg({"제목" : "count", 
                                                 "좋아요수" : "sum"}).sort_values("제목", ascending=False).reset_index()

# 가수별 좋아요 수
df_singer_like = df_ballad.groupby("가수").agg({"제목" : "count", 
                                                 "좋아요수" : "sum"}).sort_values("좋아요수", ascending=False).reset_index()
display(df_singer_soundtrack.head())
df_singer_like.head()

In [ ]:
# 곡 수 top 20 시각화
sns.barplot(df_singer_soundtrack[:20], x="제목", y="가수", palette = sns.color_palette("pastel"))

In [ ]:
# 좋아요 수 top 20 시각화
sns.barplot(df_singer_like[:20], x="좋아요수", y="가수", palette = sns.color_palette("pastel"))

In [ ]:
# 가수별 곡 수의 분포
df_singer_soundtrack["제목"].hist(bins=50).set_title("가수별 곡 수 분포");

- 인기순에 노래가 많이 오른 가수들의 노래는 대중들로부터 인정을 받았다고 볼 수 있기 때문에 이 곡들의 가사로 학습해봐도 괜찮지 않을까?
- 혹은 음악의 좋아요 수가 많은 가수들의 노래?
- 이 두 가지를 혼합한 기준?

### 연도별 분석

In [ ]:
# 연도별 인기순 데이터
sns.lineplot(data=df_ballad.groupby("년")[["제목"]].count().reset_index(), x="년", y="제목")

- 발라드 장르 "최신곡"의 인기순 정렬이다보니 최신으로 갈수록 데이터가 많아짐(1~15000곡 기준)

### 곡별 가사 길이 분포

In [ ]:
# 가사 길이 분포
df_ballad["가사"].str.len().hist(bins=50).set_title("가사 길이");

- 공백 포함 약 300~600자 사이에 가장 많이 분포함
- 가사 전처리를 러프하게 했기 때문에 실제 가사 길이와 오차는 있을 수 있음(특수문자 등)

## 맞춤법 검사기 활용 -> 사용 x
- 네이버 맞춤법 검사기 기반의 `py-hanspell` 라이브러리 사용
- 네이버 맞춤법 검사기 기반이기 때문에 500자가 넘어가면 사용하지 못하므로 문장 단위로 끊어서 사용해야함
- 사용하게 된다면 모델에 input할 가사만 맞춤법 검사해도 될듯
- 형태소 분석기 사용 시 띄어쓰기 처리 알아서 끊어주기 때문에 사용 x

In [ ]:
# spell_checker()

## 가사 형태소 분석

In [ ]:
# 형태소 분석기 호출 및 함수 정의
okt = Okt()
def okt_clean(text):
    clean_text = []
    # 품사 태깅 후 태깅 결과를 받아서 순회 
    for word in okt.pos(text, norm=True, stem=True):
        # 품사가 조사, 어미, 구두점이면 제외하고 append 로 인덱스 0번 값만 다시 리스트에 담아줌
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])
    # 공백 문자로 연결
    return " ".join(clean_text)

In [ ]:
# 판다스에서 tqdm 기능 사용하기 위해 선언
tqdm.pandas()

In [ ]:
# 모든 가사에 적용
# df_ballad["가사_전처리"] = df_ballad["가사"].progress_map(okt_clean)

### 계절별 분석
- 사랑에 미쳐버린 노래들..
- 계절별로 노래 가사 단어 빈도의 뚜렷한 차이는 발견하지 못했음

In [ ]:
# 각 계절별 좋아요수 상위 20개 추출
df_ballad_spring = df_ballad[df_ballad["계절"] == "봄"].sort_values("좋아요수", ascending=False).head(50)
df_ballad_summer = df_ballad[df_ballad["계절"] == "여름"].sort_values("좋아요수", ascending=False).head(50)
df_ballad_fall = df_ballad[df_ballad["계절"] == "가을"].sort_values("좋아요수", ascending=False).head(50)
df_ballad_winter = df_ballad[df_ballad["계절"] == "겨울"].sort_values("좋아요수", ascending=False).head(50)

In [ ]:
# 하나의 문자열로 추출
lyrics_spring = ' '.join(df_ballad_spring["가사"])
lyrics_summer = ' '.join(df_ballad_summer["가사"])
lyrics_fall = ' '.join(df_ballad_fall["가사"])
lyrics_winter = ' '.join(df_ballad_winter["가사"])

In [ ]:
# 봄 빈도수 상위 30개 명사 추출(너, 나, 내 같이 의미없는 한 자 단어가 많아서 한글자는 제외함)
lyrics_spring_noun = okt.nouns(lyrics_spring)
count_spring = Counter(lyrics_spring_noun)
count_spring_dict = dict(count_spring.most_common(100))
count_spring_df = pd.DataFrame(count_spring_dict.values(), count_spring_dict.keys())
count_spring_df = count_spring_df.reset_index()
count_spring_df.columns = ["단어", "빈도"]
count_spring_df = count_spring_df[count_spring_df["단어"].str.len() != 1].head(30)

In [ ]:
# 봄 시각화
plt.figure(figsize=(12,10))
plt.title("봄에 출시된 노래의 가사 단어 빈도수")
sns.barplot(data=count_spring_df, x="빈도", y="단어", palette = sns.color_palette("pastel"));

In [ ]:
lyrics_summer_noun = okt.nouns(lyrics_summer)
count_summer = Counter(lyrics_summer_noun)
count_summer_dict = dict(count_summer.most_common(100))
count_summer_df = pd.DataFrame(count_summer_dict.values(), count_summer_dict.keys())
count_summer_df = count_summer_df.reset_index()
count_summer_df.columns = ["단어", "빈도"]
count_summer_df = count_summer_df[count_summer_df["단어"].str.len() != 1].head(30)

In [ ]:
# 여름 시각화
plt.figure(figsize=(12,10))
plt.title("여름에 출시된 노래의 가사 단어 빈도수")
sns.barplot(data=count_summer_df, x="빈도", y="단어", palette = sns.color_palette("pastel"));

In [ ]:
lyrics_fall_noun = okt.nouns(lyrics_fall)
count_fall = Counter(lyrics_fall_noun)
count_fall_dict = dict(count_fall.most_common(100))
count_fall_df = pd.DataFrame(count_fall_dict.values(), count_fall_dict.keys())
count_fall_df = count_fall_df.reset_index()
count_fall_df.columns = ["단어", "빈도"]
count_fall_df = count_fall_df[count_fall_df["단어"].str.len() != 1].head(30)

In [ ]:
# 가을 시각화
plt.figure(figsize=(12,10))
plt.title("가을에 출시된 노래의 가사 단어 빈도수")
sns.barplot(data=count_fall_df, x="빈도", y="단어", palette = sns.color_palette("pastel"));

In [ ]:
lyrics_winter_noun = okt.nouns(lyrics_winter)
count_winter = Counter(lyrics_winter_noun)
count_winter_dict = dict(count_winter.most_common(100))
count_winter_df = pd.DataFrame(count_winter_dict.values(), count_winter_dict.keys())
count_winter_df = count_winter_df.reset_index()
count_winter_df.columns = ["단어", "빈도"]
count_winter_df = count_winter_df[count_winter_df["단어"].str.len() != 1].head(30)

In [ ]:
# 겨울 시각화
plt.figure(figsize=(12,10))
plt.title("겨울에 출시된 노래의 가사 단어 빈도수")
sns.barplot(data=count_fall_df, x="빈도", y="단어", palette = sns.color_palette("pastel"));

# 모델 input을 위한 전처리

In [ ]:
# 가사에 개행문자 없는 데이터 제거
# df_ballad = df_ballad[df_ballad["가사"].str.contains("\n")]

In [ ]:
# 기본 전처리
# df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("[^ㄱ-ㅎ가-힣0-9\n ]", "", x).strip()) # 한글 자음, 한글, 숫자, 개행문자만 남기고 제거
# df_ballad["가사"] = df_ballad["가사"].map(lambda x : re.sub("\s{2,}", "", x)) # 공백 2회 이상 제거
# df_ballad = df_ballad[df_ballad["가사"].map(lambda x : len(x) > 10)] # 전처리 후 빈 행이나 10자 이상이 안되는 데이터 제거
# df_ballad = df_ballad.reset_index(drop=True) # 인덱스 초기화
# df_ballad.shape